In [1]:
import os
import numpy as np
import pydicom
import scipy.misc
import scipy

In [2]:
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

In [3]:
contrast_path ='/Users/lifesailor/Desktop/project/gansup/data/1-100/001/CT(CE)/2.PG+30'
original_path = '/Users/lifesailor/Desktop/project/gansup/data/1-100/001/CT/2.PG+30/'

In [4]:
contrast_files = [file for file in os.listdir(contrast_path)]
orignal_files = [file for file in os.listdir(original_path)]

In [5]:
contrast_files.sort()
orignal_files.sort()

In [6]:
contrast_data = []

for file in contrast_files:
    data = pydicom.dcmread(os.path.join(contrast_path, file))
    contrast_data.append(data)
    
original_data = []

for file in orignal_files:
    data = pydicom.dcmread(os.path.join(original_path, file))
    original_data.append(data)

In [7]:
print(len(contrast_data), len(original_data))

30 30


In [8]:
contrast_image = [data.pixel_array for data in contrast_data]
original_image = [data.pixel_array for data in original_data]

# 1. Comparison

In [12]:
def subtraction(contrast, original):
    mask = contrast < original
    subtraction = contrast - original
    subtraction[mask] = 0
    return subtraction

In [13]:
comparision_image = []

for i in range(30):
    contrast = contrast_data[i].pixel_array
    original = original_data[i].pixel_array
    comparision_image.append(subtraction(contrast, original))

In [14]:
comparision_path ='/Users/lifesailor/Desktop/project/gansup/data/1-100/001/comparison'

# 2. save image

In [15]:
for i in range(30):
    scipy.misc.imsave(os.path.join(comparision_path, str(i) + "_" + "comparison" + '.png'), comparision_image[i])
    scipy.misc.imsave(os.path.join(comparision_path, str(i) + "_" + "original" + str(i) + '.png'), original_image[i])
    scipy.misc.imsave(os.path.join(comparision_path, str(i) + "_" + "contrast" + str(i) + '.png'), contrast_image[i])

AttributeError: module 'scipy.misc' has no attribute 'imsave'

# 3. imshow

In [16]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [17]:
def f(sequence):
    fig = plt.figure(figsize=(15, 5))

    fig.add_subplot(1, 3, 1)
    plt.imshow(original_image[sequence], cmap='gray')

    fig.add_subplot(1, 3, 2)
    plt.imshow(contrast_image[sequence], cmap='gray')

    fig.add_subplot(1, 3, 3)
    plt.imshow(comparision_image[sequence], cmap='gray')

In [18]:
interactive_plot = interactive(f, sequence=range(30))
interactive_plot

interactive(children=(Dropdown(description='sequence', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, …

# 4. crop middle

In [19]:
def crop_middle(image):
    return image[156:356, 156:356]

In [20]:
original_crop = []
for image in original_image:
    original_crop.append(crop_middle(image))
    
contrast_crop = []
for image in contrast_image:
    contrast_crop.append(crop_middle(image))
    
comparison_crop = []
for image in comparision_image:
    comparison_crop.append(crop_middle(image))

In [21]:
def f2(sequence):
    fig = plt.figure(figsize=(15, 5))

    fig.add_subplot(1, 3, 1)
    plt.imshow(original_crop[sequence], cmap='gray')

    fig.add_subplot(1, 3, 2)
    plt.imshow(contrast_crop[sequence], cmap='gray')

    fig.add_subplot(1, 3, 3)
    plt.imshow(comparison_crop[sequence], cmap='gray')

In [22]:
interactive_plot = interactive(f2, sequence=range(30))
interactive_plot

interactive(children=(Dropdown(description='sequence', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, …

## 5. Original Preprocessing

### 1. Clahe

In [23]:
import cv2

In [54]:
clahed_original_crop = []

for image in original_crop:
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(5,5))
    clahed_image = clahe.apply(image)
    clahed_image = cv2.resize(clahed_image, (200, 200))
    clahed_original_crop.append(clahed_image)

In [55]:
def f3(sequence):
    fig = plt.figure(figsize=(15, 5))

    fig.add_subplot(1, 3, 1)
    plt.imshow(original_crop[sequence], cmap='gray')
    
    fig.add_subplot(1, 3, 2)
    plt.imshow(contrast_crop[sequence], cmap='gray')
    
    fig.add_subplot(1, 3, 3)
    plt.imshow(clahed_original_crop[sequence], cmap='gray')

In [56]:
interactive_plot = interactive(f3, sequence=range(30))
interactive_plot

interactive(children=(Dropdown(description='sequence', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, …

### 2. Sharpening

In [51]:
kernel_sharpening = np.array([[-1,-1,-1], 
                              [-1, 9,-1],
                              [-1,-1,-1]])
sharpened_original_crop = []
for image in original_crop:
    sharpened = cv2.filter2D(image, -1, kernel_sharpening)
    sharpened_original_crop.append(sharpened)

In [52]:
def f3(sequence):
    fig = plt.figure(figsize=(15, 5))

    fig.add_subplot(1, 3, 1)
    plt.imshow(original_crop[sequence], cmap='gray')
    
    fig.add_subplot(1, 3, 2)
    plt.imshow(contrast_crop[sequence], cmap='gray')
    
    fig.add_subplot(1, 3, 3)
    plt.imshow(sharpened_original_crop[sequence], cmap='gray')

In [53]:
interactive_plot = interactive(f3, sequence=range(30))
interactive_plot

interactive(children=(Dropdown(description='sequence', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, …

## 6. Emphasize

### 1. Brightness

In [57]:
interactive_plot = interactive(f2, sequence=range(30))
interactive_plot

interactive(children=(Dropdown(description='sequence', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, …

In [109]:
emphasize_original_crop = []

for image in original_crop:
    percentile_95 = np.percentile(image, 95)
    emphasize = np.where(image > percentile_90, image, 0)
    new_image = image + emphasize * 0.2
    emphasize_original_crop.append(new_image)

In [110]:
def f4(sequence):
    fig = plt.figure(figsize=(15, 5))

    fig.add_subplot(1, 3, 1)
    plt.imshow(original_crop[sequence], cmap='gray')

    fig.add_subplot(1, 3, 2)
    plt.imshow(contrast_crop[sequence], cmap='gray')

    fig.add_subplot(1, 3, 3)
    plt.imshow(emphasize_original_crop[sequence], cmap='gray')

In [111]:
interactive_plot = interactive(f4, sequence=range(30))
interactive_plot

interactive(children=(Dropdown(description='sequence', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, …

- 우리의 목적은 혈관을 찾는 것이다.
    - 조영제를 넣었을 때 밝아지는 부분을 찾아야 하는 것은 아니다.

### 2. Darkness

In [131]:
darken_original_crop = []

for image in original_crop:
    percentile_05 = np.percentile(image, 8)
    emphasize = np.where(image < percentile_05, image, 0)
    new_image = image - emphasize * 0.5
    darken_original_crop.append(new_image)

In [134]:
def f5(sequence):
    fig = plt.figure(figsize=(15, 5))

    fig.add_subplot(1, 3, 1)
    plt.imshow(original_crop[sequence], cmap='gray')

    fig.add_subplot(1, 3, 2)
    plt.imshow(contrast_crop[sequence], cmap='gray')

    fig.add_subplot(1, 3, 3)
    plt.imshow(darken_original_crop[sequence], cmap='gray')

In [135]:
interactive_plot = interactive(f5, sequence=range(30))
interactive_plot

interactive(children=(Dropdown(description='sequence', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, …

### 발견

1. 조영제를 넣었을 때 밝아지는 부분이 원본 이미지에서 조금이라도 밝은 것은 아니다.
2. 원본 이미지에 어두운 부분이 있는데, 어두운 부분이 의미하는 것이 무엇인지 알고 싶다.
    - 꼭 조영제랑 연관이 없다.
3. 3에서 11까지가 가장 중요하다.
    - 각 순서별로 할 필요가 없다.